In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Import Data

In [29]:
data = pd.read_csv('E:/Coding/Kaggle/LendingClub/lending-club-loan-data/loan.csv', low_memory = False)

In [30]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# shape of the data
print('Data shape', data.shape)

Data shape (2260668, 145)


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260668 entries, 0 to 2260667
Columns: 145 entries, id to settlement_term
dtypes: float64(105), int64(4), object(36)
memory usage: 2.4+ GB


## Data Cleaning

To start, let's perform basic data cleaning steps: to remove the features with large amount of missing values.

In [33]:
# missing values
pd.options.display.max_rows = 150
missing = data.isnull().sum()
missing_ratio = missing / len(data)
missing_ratio = missing_ratio.reset_index()

# check the feature that has missing ration >= 0.2
missing_ratio = missing_ratio.rename(columns={'index': 'feature', 0: 'missing ratio'})
missing_ratio = missing_ratio.sort_values(by='missing ratio', ascending=False)
missing_ratio[missing_ratio['missing ratio'] >= 0.2]

,feature,missing ratio
0,id,1.000000
18,url,1.000000
1,member_id,1.000000
134,orig_projected_additional_accrued_interest,0.996273
131,hardship_length,0.995305
124,hardship_reason,0.995305
125,hardship_status,0.995305
126,deferral_term,0.995305
127,hardship_amount,0.995305
128,hardship_start_date,0.995305


Based on the information above, let's remove some of the features where the missing ratio is too big. In this analysis let's use the threshold of 30%.

In [34]:
# remove the features with missing ratio >= 0.3
columns = ['id','url','member_id', 'orig_projected_additional_accrued_interest', 'hardship_start_date', 
           'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 
           'hardship_dpd', 'hardship_loan_status', 'hardship_payoff_balance_amount', 
           'hardship_last_payment_amount', 'hardship_amount', 'deferral_term', 
           'hardship_status', 'hardship_reason', 'hardship_type', 'desc', 
           'sec_app_mths_since_last_major_derog', 'settlement_term', 
           'debt_settlement_flag_date', 'settlement_date', 'settlement_amount', 
           'settlement_percentage', 'settlement_status', 'sec_app_revol_util', 
           'revol_bal_joint', 'sec_app_open_act_il', 'sec_app_earliest_cr_line', 
           'sec_app_inq_last_6mths', 'sec_app_open_acc', 'sec_app_mort_acc', 
           'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 
           'sec_app_collections_12_mths_ex_med', 'verification_status_joint',
           'dti_joint', 'annual_inc_joint', 'mths_since_last_record', 
           'mths_since_recent_bc_dlq', 'mths_since_last_major_derog', 
           'mths_since_recent_revol_delinq', 'mths_since_last_delinq', 'next_pymnt_d', 
           'il_util', 'mths_since_rcnt_il', 'all_util', 'open_acc_6m', 'total_cu_tl', 
           'inq_last_12m', 'open_act_il', 'open_il_12m', 'max_bal_bc', 'open_rv_24m', 
           'open_rv_12m', 'total_bal_il', 'open_il_24m', 'inq_fi']

In [35]:
# drop features with too many missing values
data = data.drop(labels=columns, axis=1)

# save to local disk
data.to_csv('E:/Coding/Kaggle/LendingClub/lending-club-loan-data/data_clean.csv', index=False)

data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,2500,2500,2500.0,36 months,13.56,84.92,C,C1,Chef,10+ years,...,0.0,1.0,0.0,60124.0,16901.0,36500.0,18124.0,N,Cash,N
1,30000,30000,30000.0,60 months,18.94,777.23,D,D2,Postmaster,10+ years,...,0.0,1.0,0.0,372872.0,99468.0,15000.0,94072.0,N,Cash,N
2,5000,5000,5000.0,36 months,17.97,180.69,D,D1,Administrative,6 years,...,0.0,0.0,0.0,136927.0,11749.0,13800.0,10000.0,N,Cash,N
3,4000,4000,4000.0,36 months,18.94,146.51,D,D2,IT Supervisor,10+ years,...,100.0,0.0,0.0,385183.0,36151.0,5000.0,44984.0,N,Cash,N
4,30000,30000,30000.0,60 months,16.14,731.78,C,C4,Mechanic,10+ years,...,0.0,0.0,0.0,157548.0,29674.0,9300.0,32332.0,N,Cash,N


In [36]:
# basic information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260668 entries, 0 to 2260667
Data columns (total 87 columns):
loan_amnt                     int64
funded_amnt                   int64
funded_amnt_inv               float64
term                          object
int_rate                      float64
installment                   float64
grade                         object
sub_grade                     object
emp_title                     object
emp_length                    object
home_ownership                object
annual_inc                    float64
verification_status           object
issue_d                       object
loan_status                   object
pymnt_plan                    object
purpose                       object
title                         object
zip_code                      object
addr_state                    object
dti                           float64
delinq_2yrs                   float64
earliest_cr_line              object
inq_last_6mths                fl

Finally after cleaning up the features with a lot of missing data, we managed to reduce the number of features from 145 to 87 features. 